<a href="https://colab.research.google.com/github/Antonio12debug/Curso_DIO/blob/main/Sistema_Bancario_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re

class Usuario:
    def __init__(self, nome, data_nascimento, cpf, endereco):
        self.nome = nome
        self.data_nascimento = data_nascimento
        self.cpf = cpf
        self.endereco = endereco

class ContaCorrente:
    AGENCIA = "0001"
    numero_conta_sequencial = 1

    def __init__(self, usuario):
        self.agencia = ContaCorrente.AGENCIA
        self.numero_conta = ContaCorrente.numero_conta_sequencial
        ContaCorrente.numero_conta_sequencial += 1
        self.usuario = usuario
        self.saldo = 0
        self.extrato = []
        self.limite = 500
        self.numero_saques = 0
        self.limite_saques = 3

    def saque(self, *, valor):
        if valor > self.saldo:
            return "Saldo insuficiente"
        if valor > self.limite:
            return "Valor acima do limite de saque"
        if self.numero_saques >= self.limite_saques:
            return "Número de saques diários excedido"

        self.saldo -= valor
        self.extrato.append(f"Saque: -R${valor:.2f}")
        self.numero_saques += 1
        return self.saldo, self.extrato

    def deposito(self, /, valor):
        self.saldo += valor
        self.extrato.append(f"Depósito: +R${valor:.2f}")
        return self.saldo, self.extrato

    def mostrar_extrato(self):
        extrato_texto = "\n".join(self.extrato)
        return f"Saldo: R${self.saldo:.2f}\nExtrato:\n{extrato_texto}"

# Lista para armazenar usuários e contas
usuarios = []
contas = []

def validar_nome(nome):
    if re.match("^[A-Za-zÀ-ÖØ-öø-ÿ ]+$", nome):
        return True
    return False

def validar_data(data):
    if re.match("^\d{2}/\d{2}/\d{4}$", data):
        return True
    return False

def validar_cpf(cpf):
    if re.match("^\d{11}$", cpf):
        return True
    return False

def validar_endereco(endereco):
    if re.match("^[A-Za-zÀ-ÖØ-öø-ÿ ]+, \d+ - [A-Za-zÀ-ÖØ-öø-ÿ ]+ - [A-Za-zÀ-ÖØ-öø-ÿ ]+/[A-Za-z]{2}$", endereco):
        return True
    return False

def criar_usuario(nome, data_nascimento, cpf, endereco):
    if not validar_nome(nome):
        return "Nome inválido. Deve conter apenas letras e espaços."
    if not validar_data(data_nascimento):
        return "Data de nascimento inválida. Formato correto: dd/mm/aaaa."
    if not validar_cpf(cpf):
        return "CPF inválido. Deve conter 11 dígitos numéricos."
    if not validar_endereco(endereco):
        return "Endereço inválido. Formato correto: logradouro, nro - bairro - cidade/sigla estado."

    for usuario in usuarios:
        if usuario.cpf == cpf:
            return "Usuário com este CPF já existe"
    novo_usuario = Usuario(nome, data_nascimento, cpf, endereco)
    usuarios.append(novo_usuario)
    return "Usuário criado com sucesso"

def criar_conta_corrente(cpf):
    usuario = next((usuario for usuario in usuarios if usuario.cpf == cpf), None)
    if not usuario:
        return "Usuário não encontrado"
    nova_conta = ContaCorrente(usuario)
    contas.append(nova_conta)
    return f"Conta criada com sucesso. Agência: {nova_conta.agencia}, Conta: {nova_conta.numero_conta}"

def listar_contas():
    if not contas:
        return "Nenhuma conta encontrada."
    lista_contas = []
    for conta in contas:
        lista_contas.append(f"Agência: {conta.agencia}, Conta: {conta.numero_conta}, Usuário: {conta.usuario.nome}")
    return "\n".join(lista_contas)

def encontrar_conta(numero_conta):
    return next((conta for conta in contas if conta.numero_conta == numero_conta), None)

def encontrar_conta_por_cpf(cpf):
    for conta in contas:
        if conta.usuario.cpf == cpf:
            return conta
    return None

def menu():
    while True:
        print("\nMenu:")
        print("1. Novo usuário")
        print("2. Nova conta")
        print("3. Listar contas")
        print("4. Buscar conta existente")
        print("5. Depositar")
        print("6. Sacar")
        print("7. Extrato")
        print("8. Sair")

        opcao = input("Escolha uma opção: ")

        if opcao == "1":
            nome = input("Nome: ")
            data_nascimento = input("Data de Nascimento (dd/mm/aaaa): ")
            cpf = input("CPF: ")
            endereco = input("Endereço (logradouro, nro - bairro - cidade/sigla estado): ")
            print(criar_usuario(nome, data_nascimento, cpf, endereco))

        elif opcao == "2":
            cpf = input("CPF do usuário: ")
            print(criar_conta_corrente(cpf))

        elif opcao == "3":
            print(listar_contas())

        elif opcao == "4":
            cpf = input("CPF do usuário: ")
            conta = encontrar_conta_por_cpf(cpf)
            if conta:
                print(f"Agência: {conta.agencia}, Conta: {conta.numero_conta}, Usuário: {conta.usuario.nome}")
            else:
                print("Conta não encontrada.")

        elif opcao == "5":
            numero_conta = int(input("Número da conta: "))
            conta = encontrar_conta(numero_conta)
            if conta:
                valor = float(input("Valor do depósito: "))
                saldo, extrato = conta.deposito(valor)
                print(f"Depósito realizado. Novo saldo: R${saldo:.2f}")
            else:
                print("Conta não encontrada.")

        elif opcao == "6":
            numero_conta = int(input("Número da conta: "))
            conta = encontrar_conta(numero_conta)
            if conta:
                valor = float(input("Valor do saque: "))
                resultado = conta.saque(valor=valor)
                if isinstance(resultado, str):
                    print(resultado)
                else:
                    saldo, extrato = resultado
                    print(f"Saque realizado. Novo saldo: R${saldo:.2f}")
            else:
                print("Conta não encontrada.")

        elif opcao == "7":
            numero_conta = int(input("Número da conta: "))
            conta = encontrar_conta(numero_conta)
            if conta:
                print(conta.mostrar_extrato())
            else:
                print("Conta não encontrada.")

        elif opcao == "8":
            print("Saindo...")
            break

        else:
            print("Opção inválida. Tente novamente.")

# Adicionar dois usuários e suas contas
print(criar_usuario("João Silva", "15/04/1985", "12345678901", "Rua das Flores, 123 - Centro - Cidade/UF"))
print(criar_usuario("Maria Oliveira", "22/09/1992", "98765432102", "Avenida Brasil, 456 - Jardim - Cidade/UF"))

print(criar_conta_corrente("12345678901"))
print(criar_conta_corrente("98765432102"))

# Executar o menu
menu()
